In [1]:
import sys
import os
sys.path.append('./my_project')
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.master('local[1]').appName('my_app').getOrCreate()
import logging
logger = logging.getLogger('jupyter_logger')

# Построение архитектуры проекта при работе с PySpark

В настоящее время уже сложно найти достаточно крупную компанию, которая не использовала бы возможности накопления и использования больших данных. Одним из популярных инструментов для построения процессов обработки этих данных является pyspark. В данной статье будет представлен один из методов написания кода на pyspark таким образом, чтобы он был более читаем, легко тестируем и поддерживаем. Сразу оговорюсь, что не представляю это, как единственное правильное решение, но оно доказало свою жизнеспособность на примере того проекта, в котором я работал.

Перед тем, как перейти к конкретике, хочу немного рассказать про историю проекта, который натолкнул меня на мысли о проработке архитектуры написания кода. Большинство проектов начинаются с идеи, разрастающейся до MVP в достаточно короткие сроки. На стадии MVP основная задача продукта – показать то, что он работает и может приносить деньги. Если он этого не может, он дальше не живёт, а если может, то из него уже хотят получить готовое решение с минимальными затратами и максимальными темпами. Основная проблема тут заключается в том, что при создании MVP в код продукта вряд ли закладывались все заделы на будущее масштабирование и стабильную работу. Плюс к тому в течении эксплуатации возникает ряд вопросов и доработок для того, чтобы оно вообще работало на постоянку, а не в какой-то ограниченный промежуток времени. В общем, если даже в MVP изначально закладывалась какая-то архитектура, то он скорее не выдержит наплыва доработок и код превращается в что-то вроде наушников, которые пролежали в кармане на протяжении последнего месяца. Вроде ты и пытался сложить их аккуратненько, но доставая их концы найти весьма затруднительно. Проблема усугубляется тем, что люди, которые этот код писали, ещё более-менее в нём ориентируются, но вот для вновь прибывших сотрудников (а это не редкость при расширении проекта) погружение в такой код занимает неоправданно много времени.

Вот приблизительно на первой стадии раскатки MVP я и подключился в проект со всей его витиеватой логикой. Здесь я пропущу рассказ о моей боли, которую я испытывал при попытках составить последовательность логических операций по формированию одной таблички, которую я кропотливо собирал по 7 вкладкам кода. Скажу лишь, что именно это и подтолкнуло меня к созданию такой архитектуры, которая позволила облегчить жизнь тем, кто будет после. Надеюсь, облегчит кому-нибудь ещё. Вперёд к светлому будущему!

## Что не так?

Прежде чем решать проблему, нужно её определить. Поэтому для начала я хотел бы обозначить основные моменты, которые были самыми критичными на мой взгляд.

* **Документация.** Здесь речь идёт даже не о банальных правилах расставления комментариев и описаний функций. Они, конечно, важны, намого проще разобраться в коде, если там есть хоть какие-то пояснения, но есть и другая проблема. Наш продукт, например, находится в постоянном контакте с бизнесом. Логика его работы должна быть этому бизнесу понятна и где-то зафиксирована. При этом, естественно, в источнике, доступном бизнесу, она должна поддерживаться в актуальном состоянии. Даже если аккуратно описать все функции на понятном человеку языке, сгенерировать какой-нибудь [pydoc](https://docs.python.org/3/library/pydoc.html) или [sphinx](https://www.sphinx-doc.org/en/master/), то вряд ли вы порадуете менеджеров, если дадите им это почитать. Есть другой путь – написать менеджерам отдельное описание логики проекта. Но в таком случае при любом изменении логики (а они происходили постоянно), придётся поменять описание внутри кода, поменять описание функции, а затем ещё и переписать документацию для бизнеса. Вывод – хочется иметь такую документацию, которая была бы понятна и людям, работающим с кодом и бизнесу. А ещё неплохо бы тратить на неё не больше времени, чем на написание кода.

* **Нелинейность кода.** С одной стороны код типа ```fun(fun1(fun2…))``` очень плохо воспринимается, а ситуации, когда он прямо таки необходим в ETL процесса встречаются крайне редко. С другой стороны, возвращаемся к вопросу документации и объяснения этой логики. Чем больше вложенности, тем сложнее восстановить логику в один связный текст.

* **Недекомпозированность кода.** Работа датафреймами в pyspark ни в коем разе не останавливает полёт фантазии. Если у вас достаточно запала то написание функции можно не останавливать никогда. ```df = select.join.withColumn.withColumn.groupby.agg.select.withColumnRenamed...``` и дальше, дальше пока не оставят силы. Можно отдохнуть. А на следующий день ```df_new = df.select.join.withColumn...```. В общем, если без шуток, то я натыкался на файлы в несколько тысяч строк вот такого кода, в котором уже на 500 строчке забываешь, с чего, собственно, всё начиналось. Ещё одной большой проблемой длинного спарковского кода лично для меня является то, что очень быстро перестаёшь осознавать, какие вообще колонки есть в том или ином датафрейме.

* **Сложность тестирования.** В основном, функции содержали обращения к таблицам базы внутри себя. В таких условиях очень тяжело их хоть как-то тестировать, так как тесты на больших таблицах занимают очень много времени, а при отсутствии доступа к базе вообще невозможны, а хотелось бы. Можно, конечно, собрать свою локальную песочницу и записать все тестовые таблицы туда. Но тесты частенько помогают понять логику кода, и хотелось бы иметь тестовые таблички всегда под рукой.

Теперь, когда основные проблемы обозначены, можем перейти к примерам их решения. Далее я не хотел бы уделять время скучному разбору кода базовых классов, на которых строилось моё решение. Больше внимания я уделю основным концепциям с примерчиками использования, а сам код вы всегда сможете найти в [репозитории](https://github.com/n-surkov/PySparkPipeline).

## Структура проекта

Проекты разные, задачи разные, структуры разные. Говорить, что какая-то структура является универсальной смысла не имеет. В этом разделе просто будет приведён пример структуры, которая хорошо прижилась в нашем проекте, и в контексте которой я буду дальше вести повествование. Возьмём только минимально необходимое:

```
my_project:
|--utils
|  |--description_builder.py
|--modules
|  |--__init__.py
|  |--module_base.py
|  |--config_base.py
|  |--module1
|     |--__init__.py
|     |--config.py
|     |--module.py
|     |--runner.py
|--config.yml
|--config_sources.yml
```

На самом верхнем уровне у нас лежат конфиги проекта. У нас было всего 2 файла, поэтому в корне они никому не мешались (`config.yml`, `config_sources.yml`). Если их больше, то, конечно, можно уместить их в отдельную директорию. Дальше директория со скриптами-утилитами (`utils`) и, собственно, дерево всех модулей проекта (`modules`).

Модули проекта -- его логические части. Здесь изображена структура модулей одного уровня вложенности~~, потому что я устал рисовать~~ для простоты восприятия, но уровней может быть и больше. Главное, чтобы они были логически связаны. В каждом модуле лежит файл с конфигом этого модуля (`config.py`), файл в котором содержатся все вычисления (`module.py`) и скрипт для запуска (`runner.py`), в котором должно быть минимум кода. Файлы `module_base.py` и `config_base.py` содержат базовые классы, речь о которых пойдёт далее. Да, хранить базовые классы именно так не самое удачное решение, но оно работает, никого не беспокоит, поэтому пусть полежат здесь.

Теперь, когда мы разобрались с основой структуры, можем переходить, собственно, к самому решению, которое, на мой взгляд, сделало жизнь в проекте намного проще.

## Конфиги проекта

Начнём с конфигураций проекта. В большинстве случаев конфиги делятся на два типа:
* **статические**, которые иногда меняются, но всё-таки крайне редко
* **динамические**, которые может понадобиться менять чуть ли ни при каждом отдельном запуске скрипта. Обычно они же и передаются в качестве параметров в скриптах запуска

Начнём со статических конфигов. Обычно это словарь, полученный при чтении yaml, json или подобного файла, где задана конфигурация проекта. В нашем случае это два yaml-файла. `config.yml` -- файл, содержащий параметры, общие для всего проекта. В  нашем проекте это были сопоставления городов и макрорегионов. `config_sources.yml` -- файл, содержащий список витрин, которые используются в проекте. Источники могут быть внешними (считаются другими продуктами) и внутренними (считаются внутри проекта). Также, под проект может быть выделено несколько баз (для прода, теста, бэкапов и т.д.) и всю эту информацию нужно как-то хранить. В своём проекте мы пришли к следующей схеме описания таблиц:

```yaml
# Внешние источники
goods_tbl: 'edw_db.goods_table' # справочник товаров (ссылка)
sales_tbl: 'edw_db.sales_table' # продажи магазинов (ссылка)

# Базы данных продукта
db_backups:
    test: 'product_test_db' # база, в куторую сохраняются бэкапы расчётов
    prod: 'product_prod_db' # база, в куторую сохраняются расчёты

# Бэкапы
backups: # витрины продукта, слепок которых делается в тестовую базу
    product_table_1: # алиас таблицы
        table_name: table_name_in_database # имя таблицы в базе
        partitionedby: ['date', 'city'] # партиции
```

Сначала идёт перечисление внешних источников, которые мы только считываем. Затем список баз данных продукта, в нашем случае это продовая и тестовая базы. Затем идёт описание таблиц, расчёт которых производится в проекте. Здесь полезной дополнительной информацией будет список колонок, по которым производится партиционирование таблицы при записи.

Если говорить о динамических конфигах, то у нас они всегда передавались через bash в качестве аргументов. В силу этого, каждый скрипт запуска начинается с добавления параметров в `argparse`, причём обычно, всё это полотно задаётся кописастом из загрузчика ранее написанного модуля. В силу этого, такие параметры тоже хотелось бы хранить в нашем конфиге.

Ну и наконец, во время работы скриптов может понадобиться передавать некоторую информацию из функции в функцию. Конфиг, скорее всего будет тем объектом, который передаётся в каждую функцию и передавать параметры через него достаточно удобно.

Итого, конфиг в проекте будет объектом класса, в котором существуют 4 атрибута:
* `cfg` -- для статических конфигов из `config.yml` (считывается при инициализации объекта класса)
* `cfg_sources` -- для хранения информации по базам данных из `config_sources.yml` (считывается при инициализации объекта класса)
* `parameters` -- параметры, получающие данные при помощи `argparse`. Описание параметров, которые используются повсеместно (таких как, например, TEST -- флаг использования тестовой базы, или дата расчёта), можно задавать в `__init__` базового класса
* `tmp` -- дополнительные данные, которые могут быть добавлены в конфиг в процессе вычислений

А также несколько полезных функций:
* `getitem`, `setitem` -- для того, чтобы к классу можно было обращаться, как к словарю
* `print_description` -- для выведения описания всех конфигураций
* `add_parameter`, `parse_arguments` -- для добавления параметров и парсинга их значений из bash
* `get_table_link` -- функция, выдающая полный путь к таблице по её алиасу 

В конфиг можно также накидать и других функций, результат которых зависит от статических или динамических параметров, и которые используются повсеместно в проекте. Например, функцию, которая возвращает дату расчёта в нужном формате очень упрощает жизнь.

Ниже пример использования конфига для получения ссылки на таблицу `product_table_1` из описания выше. В конфиге по умолчанию, добавлен параметр `TEST`, который определяет базу данных проекта для проведения вычислений. Также продемонстрировано, как работать с параметрами и временными значениями конфига.

In [2]:
from modules.config_base import ConfigBase
# Пример вывода ссылки на таблицу
config = ConfigBase()
print(f"For test parameter '{config['TEST']}' table link is '{config.get_table_link('product_table_1')}'")
config.update_config({'TEST': True})
print(f"For test parameter '{config['TEST']}' table link is '{config.get_table_link('product_table_1')}'")

# Добавление параметров
config.add_parameter('logging_level', '--log_lev', type=int, default=10, nargs='?', help='Log level')
config['tmp_parameter'] = None
config.print_description()

For test parameter 'False' table link is 'product_prod_db.table_name_in_database'
For test parameter 'True' table link is 'product_test_db.table_name_in_database'
---------------------------------------------------
global_config contains:

config_parameter_1 : 12
config_parameter_2 : 
	subparam1 : param
	subparam2 : noparam
---------------------------------------------------
---------------------------------------------------
sources_config contains:

goods_tbl : edw_db.goods_table
sales_tbl : edw_db.sales_table
db_backups : 
	test : product_test_db
	prod : product_prod_db
backups : 
	product_table_1 : 
		table_name : table_name_in_database
		partitionedby : ['date', 'city']
---------------------------------------------------
---------------------------------------------------
parameters contain:

TEST : True
calc_date : 20.08.2021
logging_level : 10
---------------------------------------------------
---------------------------------------------------
temporary data contains:

tmp_par

Даже такая простая функция, как возвращение пути к таблице по её алиасу, может существенно сократить количество однотипного кода в проекте. На функцию печати содержания конфига тоже рекомендую потратить время, это экономит много времени в дальнейшем, когда модулей в проекте становится больше, чем один.

В заключении следует оговорить, что же тогда содержится в файлах `config.py` каждого отдельного модуля? В этих файлах происходит инициализация объекта базового класса `ConfigBase` и добавление в этот объект параметров, имеющих отношение к модулю (функция `add_parameter()`).

Минимальная реализация базового класса под такой конфиг находится в файле `config_base.py` [репозитория](https://github.com/n-surkov/PySparkPipeline).

## Линеаризация структуры

Иметь хороший конфиг, конечно, приятно, но [глобальных проблем](#Каким-же-должен-быть-код?) это не решает. Основной из них, на мой взгляд, является нелинейность написания кода. Для решения такого рода проблемы мне очень приглянулась идеология pipeline'ов, которые используются в pandas для построения алгоритмов обработки данных с последующим обучением моделек. В общем, каждый ETL процесс проекта мне хотелось видеть в виде последовательности шагов, выполняющихся один за другим. В моей терминологии я определил следующие два основных понятия:

* Pipeline -- ETL процесс, состоящий из последовательного выполнения шагов, принимающий на вход таблицы из БД, результатом выполнения которого является новая таблица в БД (можно несколько, но желательно небольшое количество, иначе получается много намешанной логики)
* Step -- Шаг вычислений, на вход принимает `spark.DataFrame` или обращается к базам напрямую. На выходе одна или несколько таблиц (в виде тех же `spark.DataFrame`), которые будут использоваться в дальнейших вычислениях 

Под каждый из этих пунктов был написан свой базовый класс, основные концепции которых я и попытаюсь продемонстрировать далее.

### Step

Пойдём от малого к большому и начнём с описания требований к шагу вычислений. Начнём с объёма -- шаг не должен быть большим. Обычно, в практике мы ориентировались на 100-150 строк спаркокода. Такое количество приемлемо, чтобы не потерять цепочку повествования даже при весьма заковыристой логике. Помимо этого, в каждом шаге хотелось бы иметь описание структур входных и выходных таблиц и в процессе вычислений сверять данные с этой структурой. Получаем следующие требования к объекту **Step**:

* Так как мы целимся на нормальную документацию, каждый класс-шаг должен иметь качественное описание логики его вычислений
* Должен иметь атрибуты, в которых хранятся описания схем входных и выходных таблиц
* Если у нас есть описания таблиц, то пусть он проверяет вход на соответствие описанию перед началом вычислений 
* Должен иметь функцию, в которой, собственно, и будут производиться все основные вычисления
* Должен иметь функцию `.run()`, которая будет возвращать список датафреймов, являющихся результатами вычислений. Отдельная функция тут нужна для того, чтобы проверять на соответствие описанию и выходные таблицы

Написание кода в такой парадигме будет осуществляться следующим образом:

In [5]:
from modules.module_base import StepBase
# Создание класса вычислений
class SomeStep(StepBase):
    """
    Название шага
    
    Описание вычислений класса в формате Markdown
    """
    source_tables = {  # словарь описаний источников
        'input_table_1': {  # имя, по которому можно будет обращаться к таблице при описании вычислений
            'link': 'sales_tbl',  # ссылка на таблицу (можно алиасом из config_sources, как здесь), 
                                 # 'argument' если таблица не из БД
            'description': 'Таблица для теста',  # описание таблицы
            'columns': [  # список колонок
                ('zrtpluid', 'string', 'plu', None), # имя, тип колонки в таблице, новое имя, новый тип
                ('zprice', 'double', 'price', None),
            ]
        }
    }
    
    output_tables = {
        'out_table_1': {
            'link': None,  # ссылка на таблицу (можно алиасом из config_sources), для дальнейшего сохранения
            'description': 'Таблица после теста',
            'columns': [
                ('plu', 'string'), # имя и тип колонки в выхоной таблице таблице
                ('price', 'double'),
                ('double_price', 'double'),
            ]
        }
    }
    
    def _calculations(self):
        "Вычисления класса, на выходе словарь в соответствии с self.output_tables"
        df = (
            self.tables['input_table_1']
            .withColumn('double_price', F.col('price') * 2)
            .withColumn('price', F.round('price', 0))
        )
        return {'out_table_1': df}
    
# Инициализация объекта класса
df = spark.createDataFrame([['555', 13.4]], ['zrtpluid', 'zprice'])
argument_tables = {'input_table_1': df}  # Словарь с таблицами-аргументами
step = SomeStep(spark, config, argument_tables, test=True, logger=logger)

# Запуск вычислений
result = step.run()
result['out_table_1'].show()

При этом на стадии инициализации объекта класса осуществляются:
* проверки на совпадение названий и типов колонок таблиц на входе с описанием, заданным в `source_tables`
* селект, переименование и изменение типов входных таблиц таким образом, чтобы они соответствовали описанию в `source_tables`
* складирование таблиц в переменную класа `self.tables`

На стадии запуска вычислений осуществается запуск функции `_calculations()` с последующей проверкой на то, что результат её вычислений соотносится с описанием `output_tables`.

На первый взгляд, такой подход кажется избыточно сложным. В данном примере написание простейшей операции потребовало значительных затрат. Но когда дело касается более замысловатой логики, он себя более чем оправдывает, делая погружение в такой код много более приятным. Проверки на соответствие типов входных таблиц описанию ни раз спасали, как при дебаге нового кода, так и при рефакторинге старого. Несколько спорным является момент с добавлением в описание нового названия и типа колонок входных таблиц, ведь переименовывание колонок и смена типизации нужна не всегда. Тем не менее, по моему личному ощущению это намного приятнее, чем начинать каждую новую функцию с `.withColumnRenamed` и `cast`.

Также хочется отдельно обговорить параметр `tets` при инициализации шага. Как видно в описании класса, ссылка на входную таблицу была задана алиасом из конфига. Если бы мы просто инициализировали шаг, то он бы начал стучаться в базу для того, чтобы получить схему таблицы и сравнить её с описанием. Но для тестирования нам бы хотелось просто передавать шагу на вход нужную нам таблицу. Для этого и вводится параметр `test`, который говорит шагу, что все источники переданы ему в качестве аргументов и в базы ломиться ему не нужно. 

### Pipeline

С модулем Pipeline дела обстоят много проще. Основной принцип его действия:
* создаём пустой словарь с таблицами `result`
* начинаем последовательно запускать шаги, передавая каждому шагу на вход `result` и обогащая `result` рузультатом выполнения шага
* после всех вычислений выбираем те таблицы, которые указаны в описании из `result`

Для демонстрации написания кода пайплайна можно написать ещё один шаг, который будет производить те же самые вычисления, но название входной таблицы изменится на `out_table_1`, а выходной на `out_table_2`. Я вынесу за скобки этот копипаст и перейдём к примеру:

In [10]:
from modules.module_base import PipelineBase
class Pipeline(PipelineBase):
    """
    Название пайплана
    
    Общее описание всего пайплайна вычислений
    """
    step_sequence = [
        SomeStep,
        AnotherStep
    ]
    
    output_tables = {  # Аналогично описанию в Step
        'out_table_2': {
            'link': None,
            'description': 'Финальная таблица',
            'columns': [
                ('plu', 'string'),
                ('double_price', 'double'),
            ]
        }
    }

# Инициализация пайплайна
pipeline = Pipeline(spark, config, test_arguments=argument_tables, logger=logger)

# Запуск вычислений
result = pipeline.run()
result['out_table_2'].show()

При инициализации пайплацин сначала пробегается последовательно по всем шагам и проверяет возможность вычислений исодя из описаний входных-выходных таблиц каждого шага. Коротко говоря, проверяет, соответствует ли схема выходной таблице шага "1" схеме входной таблицы шага "2", если шаг "2" пользуется результатом вычислений шага "1". Для этого не требуется даже инициализация обектов каждого шага. Если со схемами всё в порядке, то начинаются последующие вычисления в соответствии с основынми принципами, которые были представлены в начале раздела.

Важным моментом здесь является то, что в логику Pipeline закладывалось то, что он работает только с базами данных и не принимает ничего на вход. Но для теста это не удобно, поэтому нужно было предусмотреть подачу ему на вход <<игрушечных>> таблиц `test_arguments`. Если эти аргументы переданы, то при инициализации Pipeline будет запускать все свои шаги с параметром `test=True` и доступ к базам не потребуется.

Описание класса Pipeline следует хранить в файле `module.py` для каждого конкретного модуля. Описание всех шагов может не уложиться в этот файл, поэтому можно растаскать их по другим файлам, но описание именно паплайна лучше хранить для каждого модуля в файле с одинаковым названием. Далее будет показано почему это особенно удобно.

## Вкусненькое

Что же мы получаем, при использовании такой структуры?

* Во-первых, использование качественного конфига в виде класса позволяет при любых вычислениях иметь его под рукой (он является обязательным аргументом при инициализации классов типа PipelineBase и передаётся во все его шаги) и делает доступ к параметрам простым и удобным. Особенно удобно, например, описать в конфиге функцию, позволяющую преобразовывать дату вычислений (которая скорее всего будет передаваться любому скрипту как параметр) в указанный формат. В моей практике одна такая функция позволила сокртить код не менее чем на сотню строк по всему проекту.

* Во-вторых, при таком подходе сильно сокращается количество строк в файле `run.py`, который является криптом запуска модуля. Обычно, после всех импортов он выглядит как-то так:

```python
if __name__ == '__main__':
    config.parse_arguments()
    config.tune_logger(LOGGER)

    spark = load_spark()

    try:
        pipeline = module.Pipeline(spark, config, logger=LOGGER)
        result = pipeline.run()
        pipeline.save_result_to_hive(save_mode='append')
    finally:
        spark.stop()
```

* В-третьих, на своём опыте заметил, что написание кода в концепции StepBase мотивирует людей к написанию функций разумного размера. Функции величиной в пару строк писать не хочется, потому что больше времени потратишь на описание входных-выходных таблиц. Функции величиной в 500-1000 строк тоже нет большого желания писать, так как придётся описывать их логику понятным для неподготовленного читателя языком в описании класса. Таким образом получается что средний пайплайн в нашем проекте состоял из 5-7 шагов с величиной каждого шага 100-200 строк кода. Погружаться в логику работы модулей при таких обстоятельствал стало не только возможно, но и достаточно приятно.

* В-четвёртых, как мы уже успели убедиться, тестирование каждого шага является достаточно простым делом. Все таблицы, которые используются для вычислений можно достаточно просто передать на вход и не нужно ломать себе голову над тем, как вынести все инициализации `spark.table()` из тела функции.

Но это ещё не всё. Одним из самых приятных моментов, что при таком подходе очень просто собирать описание работы пайплайнов! У нас для этого есть всё:
* описание каждого шага на понятном языке (насколько это возможно)
* описание входов и выходов каждого шага
* краткое описание алгоритма в целом (его цели, время запуска и т.п. лучше как раз и указывать в описании класса Pipeline)

Описание, сгенерированное автоматически из объекта класса Pipeline, описанием которого мы занимались на протяжении статьи будет выглядеть следующим образом:

---
**Название пайплана**


Общее описание всего пайплайна вычислений

<img src="pipeline.png" width=300>

**Модуль состоит из последовательного выполнения следующих шагов:**
* **SomeStep** (Название шага):

    Описание вычислений класса в формате Markdown

  *Исходные таблицы:*

	* input_table_1 (argument) (Таблица для теста)

  *Выходные таблицы:*

	* out_table_1 (Таблица после теста)

* **AnotherStep** (Шаг, описание которого осталось за скобками):

    Делает то же, что и предыдущий:
    * удваивает цену, записывает результат в колонку `double_price`

  *Исходные таблицы:*

	* out_table_1 (Результат вычислений шага SomeStep) (Таблица после теста)

  *Выходные таблицы:*

	* out_table_2 (Таблица после второго теста)


**Результатом выполнения алгоритма являются следующие таблицы:**
* out_table_2 (Результат вычислений шага AnotherStep) (Финальная таблица):

<table>
  <thead>
  <tr>
    <th>Название колонки</th><th>Формат</th>
  </tr>
  </thead>
  <tbody>
  <tr>
    <td>plu</td><td>string</td>
  </tr>
  <tr>
    <td>double_price</td><td>double</td>
  </tr>
  </tbody>
</table>

---

Графы вычислений здесь отрисован с помощью утилиты [graphviz](https://graphviz.org). Сам код описания графа достаточно просто сгенерировать из пайплайна, аналогично документации. Печать графа в png можно осуществить при помощи утилиты `dot` на Linux иди Mac. Более того, если класс Pipeline будет приутствовать в файле `module.py` каждого модуля, то достаточно просто будет реализовать функцию, которая будет пробегать по всему проекту и в каждом модуле генерировать его описание в файл README.md. Затем это действие можно делегировать git CI и вообще забыть о том, что описание модулей нужно править где-то помимо кода проекта. Получаем качественное описание алгоритмов (с картинками, чтобы было нескучно читать), на поддержание которой в актуальном состоянии требуется минимум усилий. Профит!

## Итого

В данной статье у меня не было цели подробно разобрать концепцию пайплайнов при написании кода на спарке со всеми тонкостями её реализации. Целью являлось просто привести основные принципы, которые позволяют:

* писать код понятным и последовательным образом
* сократить время погружения в проект новых сотрудников
* писать тесты без боли
* составлять понятную лбычным людям документацию и легко поддерживать её в актуальном состоянии

Я постарался оставить в статье минимум кода, лишь для демонстрации примеров, чтобы донести основные направления мысли, которыми я руководствовался, реализуя данный функционал. Не претендую на то, что такое решение является единственно верным, но как мне, так и моим коллегам работать в такой парадигме оказалось приятно. Надеюсь, поможет это и кому-нибудь ещё.

In [7]:
class SomeStep(StepBase):
    """
    Название шага
    
    Описание вычислений класса в формате Markdown
    """
    source_tables = {  # словарь описаний источников
        'input_table_1': {  # имя, по которому можно будет обращаться к таблице при описании вычислений
            'link': 'sales_tbl',  # ссылка на таблицу (можно алиасом из config_sources, как здесь), 
                                 # 'argument' если таблица не из БД
            'description': 'Таблица для теста',  # описание таблицы
            'columns': [  # список колонок
                ('zrtpluid', 'string', 'plu', None), # имя, тип колонки в таблице, новое имя, новый тип
                ('zprice', 'double', 'price', None),
            ]
        }
    }
    
    output_tables = {
        'out_table_1': {
            'link': None,  # ссылка на таблицу (можно алиасом из config_sources), для дальнейшего сохранения
            'description': 'Таблица после теста',
            'columns': [
                ('plu', 'string'), # имя и тип колонки в выхоной таблице таблице
                ('price', 'double'),
                ('double_price', 'double'),
            ]
        }
    }
    
    def _calculations(self):
        "Вычисления класса, на выходе словарь в соответствии с self.output_tables"
        df = (
            self.tables['input_table_1']
            .withColumn('double_price', F.col('price') * 2)
            .withColumn('price', F.round('price', 0))
        )
        return {'out_table_1': df}
class AnotherStep(StepBase):
    """
    Шаг, описание которого осталос за скобками
    
    Делает то же, что и предыдущий:
    * удавивает цену, записывает результат в колонку `double_price`
    """
    source_tables = {  # словарь описаний источников
        'out_table_1': {  # имя, по которому можно будет обращаться к таблице при описании вычислений
            'link': 'argument',  # ссылка на таблицу (можно алиасом из config_sources), 
                                 # 'argument' если таблица не из БД
            'description': 'Таблица для теста',  # описание таблицы
            'columns': [  # список колонок
                ('plu', 'string', 'plu', None), # имя, тип колонки в таблице, новое имя, новый тип
                ('price', 'double', 'price', None),
            ]
        }
    }
    
    output_tables = {
        'out_table_2': {
            'link': None,  # ссылка на таблицу (можно алиасом из config_sources), для дальнейшего сохранения
            'description': 'Таблица после теста',
            'columns': [
                ('plu', 'string'), # имя и тип колонки в выхоной таблице таблице
                ('price', 'double'),
                ('double_price', 'double'),
            ]
        }
    }
    
    def _calculations(self):
        "Вычисления класса, на выходе словарь в соответствии с self.output_tables"
        df = (
            self.tables['out_table_1']
            .withColumn('double_price', F.col('price') * 2)
            .withColumn('price', F.round('price', 0))
        )
        return {'out_table_2': df}